##### Bibliotecas

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import nasdaqdatalink
import numpy_financial as npf
import yfinance as yf

In [2]:
nasdaq_api_key = 'yUkkodwj1uqL1EPc1kdS'
nasdaqdatalink.ApiConfig.api_key = nasdaq_api_key

##### Início da coleta de dados

In [7]:
start_date = '2000-01-31'
end_date = '2023-12-31'

In [35]:
selic = pd.DataFrame()
dolar = pd.DataFrame()
ibov = pd.DataFrame()

In [36]:
selic['SELIC'] = nasdaqdatalink.get('BCB/432', start_date=start_date,
                                    end_date=end_date,
                                    collapse='daily')
dolar['DOLAR'] = nasdaqdatalink.get('BCB/10813', 
                                    start_date=start_date, 
                                    end_date=end_date,
                                    collapse='daily')
ibov['IBOV'] = yf.download('^BVSP', start=start_date, end=end_date)['Adj Close']

[*********************100%***********************]  1 of 1 completed


In [37]:
selic

,SELIC
Date,
2000-01-31,19.00
2000-02-01,19.00
2000-02-02,19.00
2000-02-03,19.00
2000-02-04,19.00
...,...
2023-09-23,12.75
2023-09-24,12.75
2023-09-25,12.75


In [38]:
dolar

,DOLAR
Date,
2000-01-31,1.8016
2000-02-01,1.7924
2000-02-02,1.7892
2000-02-03,1.7771
2000-02-04,1.7777
...,...
2023-09-21,4.9223
2023-09-22,4.9125
2023-09-25,4.9600


In [39]:
ibov

,IBOV
Date,
2000-01-31,16388.000000
2000-02-01,16522.000000
2000-02-02,16868.000000
2000-02-03,17457.000000
2000-02-04,17932.000000
...,...
2023-09-22,116009.000000
2023-09-25,115925.000000
2023-09-26,114193.000000


In [33]:
macros = pd.concat([selic, dolar], 
                   axis=1)

In [40]:
macros.fillna(method='ffill', inplace=True)

In [42]:
ibov.fillna(method='ffill', inplace=True)

In [44]:
macros

,SELIC,DOLAR
Date,,
2000-01-31,19.00,1.8016
2000-02-01,19.00,1.7924
2000-02-02,19.00,1.7892
2000-02-03,19.00,1.7771
2000-02-04,19.00,1.7777
...,...,...
2023-09-23,12.75,4.9125
2023-09-24,12.75,4.9125
2023-09-25,12.75,4.9600


In [43]:
comparativo = pd.merge(ibov, macros, how='inner', left_index=True, right_index=True)
comparativo

,IBOV,SELIC,DOLAR
Date,,,
2000-01-31,16388.0,19.00,1.8016
2000-02-01,16522.0,19.00,1.7924
2000-02-02,16868.0,19.00,1.7892
2000-02-03,17457.0,19.00,1.7771
2000-02-04,17932.0,19.00,1.7777
...,...,...,...
2023-09-21,116145.0,12.75,4.9223
2023-09-22,116009.0,12.75,4.9125
2023-09-25,115925.0,12.75,4.9600


##### Plotagem do gráfico com Dolar e IBOV

In [51]:
fig = make_subplots(rows=1, cols=1, specs=[[{'secondary_y': True}]])

fig.add_trace(go.Scatter(name='DOLAR', x=comparativo.index, 
                         y=comparativo['DOLAR']), row=1, col=1)
fig.add_trace(go.Scatter(name='IBOV', x=comparativo.index, 
                         y=comparativo['IBOV']), row=1, col=1, 
              secondary_y=True)

fig.update_layout(title_text= 'Dolar vs IBOV', 
                  template='simple_white', width=1000, height=800)
fig.show()

##### Simulação de investimento

In [65]:
df = comparativo.copy(True)

In [66]:
aporte_inicial = 10000

In [67]:
pd.options.display.float_format = '{:.2f}'.format

In [68]:
df['DOLAR'] = df['DOLAR'] / df['DOLAR'].iloc[0] * aporte_inicial
df['IBOV'] = df['IBOV'] / df['IBOV'].iloc[0] * aporte_inicial
df

,IBOV,SELIC,DOLAR
Date,,,
2000-01-31,10000.00,19.00,10000.00
2000-02-01,10081.77,19.00,9948.93
2000-02-02,10292.90,19.00,9931.17
2000-02-03,10652.31,19.00,9864.01
2000-02-04,10942.15,19.00,9867.34
...,...,...,...
2023-09-21,70871.98,12.75,27321.83
2023-09-22,70788.99,12.75,27267.43
2023-09-25,70737.73,12.75,27531.08


In [75]:
taxas = {'SELIC 100%': (1 + df['SELIC'] / 100) ** (1 / 248) - 1}

In [76]:
for taxa_nome, taxa in taxas.items():
    aporte_inicial_temp = aporte_inicial
    patrimonio = [aporte_inicial_temp]
    for i in range(1, len(df)):
        aporte_inicial_temp = aporte_inicial_temp * (1 + taxa.iloc[i])
        patrimonio.append(aporte_inicial_temp)
    df[taxa_nome] = patrimonio

In [77]:
df

,IBOV,SELIC,DOLAR,SELIC 100%
Date,,,,
2000-01-31,10000.00,19.00,10000.00,10000.00
2000-02-01,10081.77,19.00,9948.93,10007.02
2000-02-02,10292.90,19.00,9931.17,10014.04
2000-02-03,10652.31,19.00,9864.01,10021.06
2000-02-04,10942.15,19.00,9867.34,10028.10
...,...,...,...,...
2023-09-21,70871.98,12.75,27321.83,152334.24
2023-09-22,70788.99,12.75,27267.43,152407.96
2023-09-25,70737.73,12.75,27531.08,152481.73


##### Gráfico de patrimônio

In [78]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(go.Scatter(name='SELIC 100%', x=df.index, 
                         y=df['SELIC 100%']), row=1, col=1)
fig.add_trace(go.Scatter(name='DOLAR', x=df.index, 
                         y=df['DOLAR']), row=1, col=1)
fig.add_trace(go.Scatter(name='IBOV', x=df.index, 
                         y=df['IBOV']), row=1, col=1)

fig.update_layout(title_text= 'Evolução do Patrimônio Investindo R$ 10.000', 
                  template='simple_white', width=1000, height=800)
fig.show()

In [81]:
df['IBOV'].iloc[-1]

69762.63

In [90]:
retorno_selic_ano = round(npf.rate(23.667, 0, -aporte_inicial, df['SELIC 100%'].iloc[-1])*100, 2)
retorno_ibov_ano = round(npf.rate(23.667, 0, -aporte_inicial, df['IBOV'].iloc[-1])*100, 2)
retorno_dolar_ano = round(npf.rate(23.667, 0, -aporte_inicial, df['DOLAR'].iloc[-1])*100, 2)

In [92]:
print(f'Retorno SELIC anual {retorno_selic_ano}%')
print(f'Retorno IBOV anual {retorno_ibov_ano}%')
print(f'Retorno DOLAR anual {retorno_dolar_ano}%')

Retorno SELIC anual 12.2%
Retorno IBOV anual 8.55%
Retorno DOLAR anual 4.43%


In [93]:
retorno_selic_mes = round(npf.rate(284, 0, -aporte_inicial, df['SELIC 100%'].iloc[-1])*100, 2)
retorno_ibov_mes = round(npf.rate(284, 0, -aporte_inicial, df['IBOV'].iloc[-1])*100, 2)
retorno_dolar_mes = round(npf.rate(284, 0, -aporte_inicial, df['DOLAR'].iloc[-1])*100, 2)

In [94]:
print(f'Retorno SELIC mensal {retorno_selic_mes}%')
print(f'Retorno IBOV mensal {retorno_ibov_mes}%')
print(f'Retorno DOLAR mensal {retorno_dolar_mes}%')

Retorno SELIC mensal 0.96%
Retorno IBOV mensal 0.69%
Retorno DOLAR mensal 0.36%
